#Importing libraries and data

In [ ]:
! pip install osmnx
! pip install geopandas
!pip install pygeos

In [43]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ast import literal_eval
import osmnx as ox
from shapely.geometry import Polygon
import seaborn as sns
import numpy as np
from shapely.geometry import Point, LineString, shape

In [ ]:
!python -m pip uninstall matplotlib
!!pip install matplotlib==3.1.3

In [69]:
# reading road construction polygons
road_construction = pd.read_csv('hungary_road_construction.csv',sep=";",index_col=None,header=None)
road_construction.columns = ["name","year","month","day","points"]
road_construction.head()

,name,year,month,day,points
0,Polgári Tisza-hid,2002,10.0,1.0,"[[21.0552074392,47.8535623743],[21.0510017355,..."
1,Szekszard Szent Laszlo-hid,2003,7.0,4.0,"[[18.8875748594,46.3498790197],[18.887274452,4..."
2,Dunaujvaros Pentele-hid,2007,7.0,23.0,"[[18.9884702044,46.8892504416],[18.9010945635,..."
3,M3 Fuzesabony-Polgar,2002,NaN,NaN,"[[20.4282135249,47.7251911319],[20.4055542231,..."
4,M3 Polgar-Gorbehaza,2004,9.0,NaN,"[[21.2596564086,47.7818136227],[21.1813788207,..."


 we don't need the columns month and day so I will remove them!

In [70]:
road_construction.drop(columns=['month' , 'day'] , inplace=True)
#Years were string I changed them to int
road_construction.year=road_construction.year.map(int)
# converting list of list strings to actual Python objects
road_construction["points"] = road_construction["points"].map(literal_eval)

In [ ]:
#Load the data that Eszter prepared
Gs = ox.io.load_graphml(filepath='hungary_main_roads.graphml')
nodes, edges = ox.graph_to_gdfs(Gs)

In [105]:
# Convert that to geodataframe and use shapley to convert roads to line string
gdf = gpd.GeoDataFrame(road_construction, crs={'init': 'EPSG:4326'}, geometry=road_construction['points'].apply(lambda x: LineString(x)))
#delete column which is not needed
gdf.drop(columns=['points'], inplace=True)
#Join left the constructions to bring the year column
intersections = gpd.sjoin(edges, gdf, how='left')
#Values that are NAN to -100000 to make it workable
intersections.year.fillna(-100000, inplace=True)
#make a new graph
GG=ox.graph_from_gdfs(nodes, intersections)

In [119]:
!mkdir data

In [ ]:
import matplotlib as mpl
cmap = mpl.cm.get_cmap('viridis')
norm = mpl.colors.Normalize(vmin=50,vmax=130)
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
for j in range(2002, 2013):
  fig, ax = ox.plot_graph(GG,node_color='k',edge_color=['r' if intersections.iloc[i].year>j else intersections.iloc[i].speedcolor for i in range(len(intersections))], show=False, edge_linewidth=10, bgcolor = 'white', figsize=(30, 20))
  cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),shrink=0.65)
  cbar.set_label('Route Max speed', fontsize = 30)
  cbar.ax.tick_params(labelsize=25)
  ax.set_title('Year %s'%j, fontsize=50)
  fig.savefig('data/%s.png'%j, dpi=100 )

creat .gif for our work

In [152]:
import imageio
images = []
for year in range(2002 ,2013):
    images.append(imageio.imread('data/%s.png'%year))
imageio.mimsave('movie.gif', images, duration=1 )

In [ ]:
!zip -r /content/hungary_road_construction.zip /content/data